# Creación Variables del Pasivo

In [1]:
# cuentas_out['prom_dias_desde_ult_trans'] = gp_cuentas['dias_desde_ultima_trans'].mean()
# cuentas_out['prom_antiguedad_cuenta'] = gp_cuentas['antiguedad'].mean()

In [2]:
import pandas as pd 
import numpy as np 

pd.set_option('max_columns',None)
#pd.set_option('max_rows',100)

from datetime import datetime, timedelta 
from dateutil import relativedelta

#pd.set_option('max_columns',False)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# path_in="s3://adl-refined-dev-popular/propension/raw_piloto/producto/pasivo/2018/"
# #path_in = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"
# path_out = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"


# for i in range(1,4):
    
#     if i<=9: var_aux = '0'+str(i)
#     else: var_aux = str(i)
        
#     data = pd.read_csv(path_in+var_aux+".csv",sep=",",encoding='utf_8',
#                         dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
    
#     print('Periodo 2020',var_aux)
#     print(data.shape)
    
#     data.to_csv(path_out+var_aux+"2.csv",sep="|",encoding='utf_8', index= False)
    
#     del data

In [4]:
# def func_dates(tabla_final,date):
#     if date == '201907':
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], dayfirst= True)
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], dayfirst= True)
#     elif date in dates2:
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%d/%m/%Y')
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%d/%m/%Y')
#     else:
#         tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y%m%d',errors='coerce')
#         tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y%m%d',errors='coerce')
        
#     return tabla_final

## Creando pasivos Total

In [5]:
# path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"

# str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
#             '202001','202002','202003','202003']

# # str_dates = ['202004']

# product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}


# dates1 = ['201904','201908','201909','201910','201911','201912','202001','202002','202003','202004','202005','202006']
# dates2 = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#          '201901','201902','201903','201905','201906','201907']

# for ix, date in enumerate(str_dates):
    
#     print('Periodo:',date)
#     if ix == 0:
#         tabla_final = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
#                     dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
#         tabla_final.columns = tabla_final.columns.str.lower()
        
#         ## Cambiar tipo producto:
#         tabla_final['periodo'] = int(date)
#         tabla_final['tipo_producto'] = tabla_final['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
#         tabla_final = func_dates(tabla_final,date)
        
#         print(tabla_final.shape)  
        
#     else:
        
#         tabla_temp = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
#                     dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
#         tabla_temp.columns = tabla_temp.columns.str.lower()
        
#         ## Cambiar tipo producto:
#         tabla_temp['periodo'] = int(date)
#         tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
#         tabla_temp = func_dates(tabla_temp,date)
        
#         print(tabla_temp.shape)
        
#         tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)
        
        
#         del tabla_temp

In [6]:
# tabla_final2 = tabla_final.copy() 

In [7]:
import s3fs
import pyarrow.parquet as pq

fs = s3fs.S3FileSystem()

def read_parquet(date):

    input_path = path_in+'productos_pasivo_aaatr_M'+date
    input = input_path
    dataset = pq.ParquetDataset(input, filesystem=fs)
    lib = dataset.read()
    lib = lib.to_pandas()

    del dataset
    
    return lib


In [8]:
path_in="s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo/"

# str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
#             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
#             '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012']

str_dates = ['202102','202103','202104']


product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}


for ix, date in enumerate(str_dates):
    
    tabla_temp = read_parquet(date)
    tabla_temp.columns = tabla_temp.columns.str.lower()

    ## Cambiar tipo producto:
    tabla_temp['periodo'] = int(date)
    tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)

    print(tabla_temp.shape)
    
    print('Periodo:',date)
    if ix == 0:
        tabla_final = tabla_temp
    else:
        tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)
         
    del tabla_temp

/home/jupyter/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


(693024, 17)
Periodo: 202012
(686872, 17)
Periodo: 202101
(685895, 17)
Periodo: 202102


In [9]:
## Cast dates
tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'])
tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'])

In [10]:
# tabla_final2 = tabla_final2.drop_duplicates()
# tabla_final = tabla_final.drop_duplicates()

# tabla_final = pd.concat([tabla_final,tabla_final2], ignore_index=True)

In [11]:
tabla_final.columns

Index(['sk_cliente', 'id_cliente', 'tipo_id', 'cod_producto', 'nro_cuenta',
       'fecha_ultima_transaccion', 'fecha_apertura', 'estado', 'saldo',
       'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5',
       'promedio6', 'periodo', 'tipo_producto'],
      dtype='object')

In [12]:
tabla_final.reset_index(inplace=True)

In [13]:
tabla_final = tabla_final.drop(columns={'index'})

In [14]:
print(tabla_final.shape)
tabla_final

(2065791, 17)


,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
0,589512,193655260482175901,1,230,230470139130,2020-08-05,2009-09-21,Activa,4734.0,44,4734,4734.0,4734,13496.0,27817.0,202012,ahorro
1,383846,176655242662434501,1,230,230269037354,2020-11-11,2019-03-11,Activa,9297.0,43,9297,86939.0,775019,2529867.0,1727435.0,202012,ahorro
2,None,None,1,230,230040255697,2021-01-04,2020-05-04,Activa,266.0,52,215273,124869.0,91883,112030.0,119374.0,202012,ahorro
3,1879480,628654886748163501,1,230,230364696526,2020-07-28,2018-12-13,Activa,13002.0,15,13002,13002.0,13002,21763.0,33351.0,202012,ahorro
4,3531065,463556583560250501,1,230,230353075104,2020-12-28,2019-06-17,Activa,639.0,57,6613,46606.0,37181,23836.0,3480.0,202012,ahorro
5,None,None,1,230,230098057920,2020-12-24,2019-12-10,Activa,8370.0,73,19794,159311.0,21052,8783.0,9952.0,202012,ahorro
6,None,None,1,230,230098057144,2021-01-05,2019-12-10,Activa,153473.0,76,101264,96966.0,132870,167686.0,39083.0,202012,ahorro
7,3524680,117956583634699602,1,230,230362350845,2021-01-04,2019-05-29,Activa,783598.0,41,558523,152489.0,4128,11892.0,309150.0,202012,ahorro
8,3513263,135456584306096301,1,230,230098048994,2020-12-22,2019-06-11,Activa,4520.0,67,12229,70751.0,47195,2136.0,9673.0,202012,ahorro
9,None,None,1,230,230026181784,2021-01-02,2019-12-16,Activa,9727.0,17,66898,132743.0,66185,83451.0,40072.0,202012,ahorro


In [15]:
tabla_final.count()

sk_cliente                  1967547
id_cliente                  1967547
tipo_id                     2065791
cod_producto                2065791
nro_cuenta                  2065791
fecha_ultima_transaccion    2032805
fecha_apertura              2065782
estado                      2065791
saldo                       2065791
promedio1                   2065791
promedio2                   2065791
promedio3                   2065791
promedio4                   2065791
promedio5                   2065791
promedio6                   2065791
periodo                     2065791
tipo_producto               2065791
dtype: int64

In [16]:
tabla_final.tail()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
2065786,2618473,758952307367168402,1,230,230470064189,2021-02-25,2006-04-03,Activa,7748.0,11,79842,240227.0,1448281,786386.0,474419.0,202102,ahorro
2065787,2483365,725955242634580402,1,230,230470202490,2021-03-01,2019-03-06,Activa,9007738.0,21,7324854,3120290.0,2823995,1880848.0,2096519.0,202102,ahorro
2065788,2756764,862252308256137301,1,230,230251018529,2021-02-24,2005-03-15,Activa,1395469.0,18,299492,320279.0,356540,674589.0,310739.0,202102,ahorro
2065789,3115603,881452305336560101,1,300,30018002817195,1997-03-07,2021-02-19,Depositado,1500000.0,535714,29,0.0,0,0.0,0.0,202102,cdt
2065790,3115603,881452305336560101,1,300,30018002822823,1997-03-07,2015-10-06,Depositado,600000.0,600000,0,0.0,0,0.0,0.0,202102,cdt


In [17]:
tabla_final.dtypes

sk_cliente                          object
id_cliente                          object
tipo_id                             object
cod_producto                        object
nro_cuenta                          object
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                              float64
promedio1                            int64
promedio2                            int64
promedio3                          float64
promedio4                            int64
promedio5                          float64
promedio6                          float64
periodo                              int64
tipo_producto                       object
dtype: object

In [18]:
tabla_final.groupby('periodo').size()

periodo
202012    693024
202101    686872
202102    685895
dtype: int64

In [19]:
## eliminando comas:

# tabla_final['saldo'] = [float(str(x).replace(',','.')) for x in tabla_final['saldo']]
# tabla_final['promedio1'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio1']]
# tabla_final['promedio2'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio2']]
# tabla_final['promedio3'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio3']]
# tabla_final['promedio4'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio4']]
# tabla_final['promedio5'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio5']]
# tabla_final['promedio6'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio6']]


In [20]:
tabla_final.head()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
0,589512,193655260482175901,1,230,230470139130,2020-08-05,2009-09-21,Activa,4734.0,44,4734,4734.0,4734,13496.0,27817.0,202012,ahorro
1,383846,176655242662434501,1,230,230269037354,2020-11-11,2019-03-11,Activa,9297.0,43,9297,86939.0,775019,2529867.0,1727435.0,202012,ahorro
2,None,None,1,230,230040255697,2021-01-04,2020-05-04,Activa,266.0,52,215273,124869.0,91883,112030.0,119374.0,202012,ahorro
3,1879480,628654886748163501,1,230,230364696526,2020-07-28,2018-12-13,Activa,13002.0,15,13002,13002.0,13002,21763.0,33351.0,202012,ahorro
4,3531065,463556583560250501,1,230,230353075104,2020-12-28,2019-06-17,Activa,639.0,57,6613,46606.0,37181,23836.0,3480.0,202012,ahorro


In [21]:
tabla_final.tail()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
2065786,2618473,758952307367168402,1,230,230470064189,2021-02-25,2006-04-03,Activa,7748.0,11,79842,240227.0,1448281,786386.0,474419.0,202102,ahorro
2065787,2483365,725955242634580402,1,230,230470202490,2021-03-01,2019-03-06,Activa,9007738.0,21,7324854,3120290.0,2823995,1880848.0,2096519.0,202102,ahorro
2065788,2756764,862252308256137301,1,230,230251018529,2021-02-24,2005-03-15,Activa,1395469.0,18,299492,320279.0,356540,674589.0,310739.0,202102,ahorro
2065789,3115603,881452305336560101,1,300,30018002817195,1997-03-07,2021-02-19,Depositado,1500000.0,535714,29,0.0,0,0.0,0.0,202102,cdt
2065790,3115603,881452305336560101,1,300,30018002822823,1997-03-07,2015-10-06,Depositado,600000.0,600000,0,0.0,0,0.0,0.0,202102,cdt


In [22]:
## Cast variables

tabla_final = tabla_final[tabla_final['id_cliente'].notnull()]
tabla_final = tabla_final[tabla_final['nro_cuenta'].isna()==False]

tabla_final['id_cliente'] = tabla_final['id_cliente'].astype('int64')
tabla_final['nro_cuenta'] = tabla_final['nro_cuenta'].astype('int64')
tabla_final['cod_producto'] = tabla_final['cod_producto'].astype('int64')

# tabla_final['saldo'] = tabla_final['saldo'].astype('float64')
# tabla_final['promedio1'] = tabla_final['promedio1'].astype('float64')
# tabla_final['promedio2'] = tabla_final['promedio2'].astype('float64')
# tabla_final['promedio3'] = tabla_final['promedio3'].astype('float64')
# tabla_final['promedio4'] = tabla_final['promedio4'].astype('float64')
# tabla_final['promedio5'] = tabla_final['promedio5'].astype('float64')
# tabla_final['promedio6'] = tabla_final['promedio6'].astype('float64')
tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y-%m-%d')
tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y-%m-%d')

In [23]:
tabla_final[tabla_final['periodo'] == 202102].count()

sk_cliente                  652111
id_cliente                  652111
tipo_id                     652111
cod_producto                652111
nro_cuenta                  652111
fecha_ultima_transaccion    641695
fecha_apertura              652108
estado                      652111
saldo                       652111
promedio1                   652111
promedio2                   652111
promedio3                   652111
promedio4                   652111
promedio5                   652111
promedio6                   652111
periodo                     652111
tipo_producto               652111
dtype: int64

In [24]:
tabla_final.dtypes

sk_cliente                          object
id_cliente                           int64
tipo_id                             object
cod_producto                         int64
nro_cuenta                           int64
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                              float64
promedio1                            int64
promedio2                            int64
promedio3                          float64
promedio4                            int64
promedio5                          float64
promedio6                          float64
periodo                              int64
tipo_producto                       object
dtype: object

In [25]:
tabla_final[tabla_final['periodo'] == 202102].head()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
1379896,589512,193655260482175901,1,230,230470139130,2021-02-28,2009-09-21,Activa,4041523.0,2703615,4734,4734.0,4734,4734.0,13496.0,202102,ahorro
1379897,383846,176655242662434501,1,230,230269037354,2021-02-22,2019-03-11,Activa,329298.0,6,89297,9297.0,9297,86939.0,775019.0,202102,ahorro
1379899,3531065,463556583560250501,1,230,230353075104,2021-02-27,2019-06-17,Activa,8855.0,65,34987,70401.0,6613,46606.0,37181.0,202102,ahorro
1379902,3524680,117956583634699602,1,230,230362350845,2021-02-28,2019-05-29,Activa,953260.0,97,1087687,861495.0,558523,152489.0,4128.0,202102,ahorro
1379903,3513263,135456584306096301,1,230,230098048994,2021-01-23,2019-06-11,Activa,2484.0,72,2484,101035.0,12229,70751.0,47195.0,202102,ahorro


In [26]:
tabla_final = tabla_final[tabla_final['id_cliente'] != -1]

In [27]:
tabla_final['fecha'] = pd.to_datetime(tabla_final['periodo'], format='%Y%m')

In [28]:
periodo = 202104

In [29]:
path_in="s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_tot_M"

tabla_final.to_parquet(path_in+str(periodo), engine='pyarrow',index=False)

# Cargando Data

In [30]:
import gc
gc.collect()

235

In [31]:
path_in = "s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_tot_M"
periodo = 202102

In [32]:
pasivo_201803_202003 = pd.read_parquet(path_in+str(periodo), engine='pyarrow')

/home/jupyter/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/jupyter/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)


In [33]:
pasivo_201803_202003.head()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto,fecha
0,589512,193655260482175901,1,230,230470139130,2020-08-05,2009-09-21,Activa,4734.0,44,4734,4734.0,4734,13496.0,27817.0,202012,ahorro,2020-12-01
1,383846,176655242662434501,1,230,230269037354,2020-11-11,2019-03-11,Activa,9297.0,43,9297,86939.0,775019,2529867.0,1727435.0,202012,ahorro,2020-12-01
2,1879480,628654886748163501,1,230,230364696526,2020-07-28,2018-12-13,Activa,13002.0,15,13002,13002.0,13002,21763.0,33351.0,202012,ahorro,2020-12-01
3,3531065,463556583560250501,1,230,230353075104,2020-12-28,2019-06-17,Activa,639.0,57,6613,46606.0,37181,23836.0,3480.0,202012,ahorro,2020-12-01
4,3524680,117956583634699602,1,230,230362350845,2021-01-04,2019-05-29,Activa,783598.0,41,558523,152489.0,4128,11892.0,309150.0,202012,ahorro,2020-12-01


In [34]:
## Validación de duplicados a nivel 'id_cliente','nrooblig','fecha'

In [35]:
pasivo_201803_202003=pasivo_201803_202003.sort_values(['id_cliente','nro_cuenta','periodo'])

In [36]:
#Se eliminan duplicados
pasivo_201803_202003=pasivo_201803_202003.drop_duplicates(subset=['id_cliente','nro_cuenta','periodo'],keep='first')

In [37]:
pasivo_201803_202003.shape

(1963902, 18)

In [38]:
pasivo_201803_202003['fecha_pasivo']=pasivo_201803_202003['periodo'].astype(int)

In [39]:
##Creacion variables SMLV

In [40]:
pasivo_201803_202003['ano_po'] = pasivo_201803_202003['periodo'].astype('str').str[0:4].astype(int)

#Se asigna el salario minimo mensual vigente de 2018 y 2019
pasivo_201803_202003['SMLV'] = np.where(pasivo_201803_202003['ano_po'] == 2018, 781242,
                                        np.where(pasivo_201803_202003['ano_po'] == 2019, 828116,
                                                 np.where(pasivo_201803_202003['ano_po'] == 2020, 877803,
                                                          np.where(pasivo_201803_202003['ano_po'] == 2021, 908526,
                                                          np.nan))))


In [41]:
pasivo_201803_202003['dia_fecha_corte'] = pasivo_201803_202003['fecha']+timedelta(days=37) ##TODO: Confirmar días

In [42]:
pasivo_201803_202003

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto,fecha,fecha_pasivo,ano_po,SMLV,dia_fecha_corte
626623,440856,101052294083942902,3,250,250180964801,2010-02-19,2006-06-16,Activa,0.0,0,0,0.0,0,0.0,0.0,202012,ahorro,2020-12-01,202012,2020,877803.0,2021-01-07
1281605,440856,101052294083942902,3,250,250180964801,2010-02-19,2006-06-16,Activa,0.0,0,0,0.0,0,0.0,0.0,202101,ahorro,2021-01-01,202101,2021,908526.0,2021-02-07
1933793,440856,101052294083942902,3,250,250180964801,2010-02-19,2006-06-16,Activa,0.0,0,0,0.0,0,0.0,0.0,202102,ahorro,2021-02-01,202102,2021,908526.0,2021-03-10
578282,491822,101052314032556001,1,230,230034135194,2020-10-15,2020-09-21,Activa,3745.0,75,3745,3745.0,1543542,14998348.0,0.0,202012,ahorro,2020-12-01,202012,2020,877803.0,2021-01-07
1233696,491822,101052314032556001,1,230,230034135194,2020-10-15,2020-09-21,Activa,3745.0,75,3745,3745.0,3745,1543542.0,14998348.0,202101,ahorro,2021-01-01,202101,2021,908526.0,2021-02-07
1885995,491822,101052314032556001,1,230,230034135194,2020-10-15,2020-09-21,Activa,3745.0,75,3745,3745.0,3745,3745.0,1543542.0,202102,ahorro,2021-02-01,202102,2021,908526.0,2021-03-10
493051,285985,101052315812399301,1,230,230015162027,2020-12-18,2016-02-17,Activa,675466.0,79,507040,43504.0,305,47873.0,47178.0,202012,ahorro,2020-12-01,202012,2020,877803.0,2021-01-07
1149032,285985,101052315812399301,1,230,230015162027,2021-01-18,2016-02-17,Activa,5489.0,9,276130,507040.0,43504,305.0,47873.0,202101,ahorro,2021-01-01,202101,2021,908526.0,2021-02-07
1801590,285985,101052315812399301,1,230,230015162027,2021-01-18,2016-02-17,Activa,5489.0,9,5489,276130.0,507040,43504.0,305.0,202102,ahorro,2021-02-01,202102,2021,908526.0,2021-03-10
427711,441894,101055253543395201,1,230,230070100219,2020-12-24,2006-10-06,Activa,12223412.0,24,11213426,11095224.0,7964281,6404090.0,5764757.0,202012,ahorro,2020-12-01,202012,2020,877803.0,2021-01-07


### Creación base a nivel id_cliente, fecha

In [43]:
pasivo_201803_202003_cli_fec=pd.DataFrame(pasivo_201803_202003.groupby(['id_cliente','periodo'])[['SMLV','fecha_pasivo']].max())

In [44]:
pasivo_201803_202003_cli_fec.shape

(1847224, 2)

In [45]:
pasivo_201803_202003_cli_fec.head()

SMLV  fecha_pasivo
id_cliente         periodo                        
101052294083942902 202012   877803.0        202012
                   202101   908526.0        202101
                   202102   908526.0        202102
101052314032556001 202012   877803.0        202012
                   202101   908526.0        202101

### Variables Pasivo

In [46]:
# Número de cuentas totales vigentes al punto de observación
# Número de cuentas vigentes al punto de observación tipo cta Y
# Dias desde la última transacción
# Antigüedad del cliente con ctas del pasivo
# meses desde la última apertura de cta
# Saldo total a fin de mes
# Saldo maximo a fin de mes
# Saldo mínimo a fin de mes
# Saldo total x meses atrás
# Saldo promedio últimos x meses
# Saldo promedio últimos x meses por tipo cta Y
# tipo de cuenta Digital, nueva, etc.

In [47]:
pasivo_201803_202003.columns

Index(['sk_cliente', 'id_cliente', 'tipo_id', 'cod_producto', 'nro_cuenta',
       'fecha_ultima_transaccion', 'fecha_apertura', 'estado', 'saldo',
       'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5',
       'promedio6', 'periodo', 'tipo_producto', 'fecha', 'fecha_pasivo',
       'ano_po', 'SMLV', 'dia_fecha_corte'],
      dtype='object')

In [48]:
## Agrupaciones
gp_pasv = pasivo_201803_202003.groupby(['id_cliente','periodo'])
# gp_pasv_cc = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='corriente'].groupby(['id_cliente','periodo'])
gp_pasv_ca = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='ahorro'].groupby(['id_cliente','periodo'])
# gp_pasv_cdt = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='cdt'].groupby(['id_cliente','periodo'])

In [49]:
## Antiguedad Total
pasivo_201803_202003_cli_fec['pasv_antig_total'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].min())/np.timedelta64(30, 'D')
## Numero meses ultima apertura
pasivo_201803_202003_cli_fec['pasv_num_meses_ult_apertura'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].max())/np.timedelta64(30, 'D')
## Dia última transacción
pasivo_201803_202003_cli_fec['pasv_dias_desde_ultima_trans'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_ultima_transaccion'].max())/np.timedelta64(1, 'D')
#

## Saldo Total por cartera fin de mes
pasivo_201803_202003_cli_fec['pasv_saldo_tot_fin_mes_smlv'] = (gp_pasv['saldo'].sum())/(gp_pasv['SMLV'].max())
# pasivo_201803_202003_cli_fec['pasv_saldo_cc_fin_mes_smlv'] = (gp_pasv_cc['saldo'].sum())/(gp_pasv_cc['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].sum())/(gp_pasv_ca['SMLV'].max())
# pasivo_201803_202003_cli_fec['pasv_saldo_cdt_fin_mes_smlv'] = (gp_pasv_cdt['saldo'].sum())/(gp_pasv_cdt['SMLV'].max())

# Creando fecha
pasivo_201803_202003_cli_fec.reset_index(inplace=True)
pasivo_201803_202003_cli_fec['fecha'] = pd.to_datetime(pasivo_201803_202003_cli_fec['periodo'], format='%Y%m')

In [50]:
def gp_func(ids, date):
    fecha = pd.to_datetime(date, format='%Y%m')
    n_meses = [1]
    
    pasv_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    
    for n in n_meses:
        col_name = 'pasv_saldo_total_'+str(n)+'mes_atras'
#         col_name_cc = 'pasv_saldo_cc_'+str(n)+'mes_atras'
        col_name_ca = 'pasv_saldo_ca_'+str(n)+'mes_atras'
#         col_name_cdt = 'pasv_saldo_cdt_'+str(n)+'mes_atras'
        date_past = fecha - timedelta(days=n*30)

        gp = pasivo_201803_202003_cli_fec[(pasivo_201803_202003_cli_fec['fecha']>date_past)&(pasivo_201803_202003_cli_fec['fecha']<=fecha)].groupby('id_cliente')
        
        pasv_out[col_name] = gp['pasv_saldo_tot_fin_mes_smlv'].sum()/n
#         pasv_out[col_name_cc] = gp['pasv_saldo_cc_fin_mes_smlv'].sum()/n
        pasv_out[col_name_ca] = gp['pasv_saldo_ca_fin_mes_smlv'].sum()/n
#         pasv_out[col_name_cdt] = gp['pasv_saldo_cdt_fin_mes_smlv'].sum()/n
        
    pasv_out['periodo'] = int(date)
    pasv_out.index.rename('id_cliente', inplace=True)
    pasv_out.reset_index(inplace=True)
    
    return pasv_out

In [51]:
str_dates = ['202012','202101','202102']

for ix, date in enumerate(str_dates):
    print(date)
    ids = pasivo_201803_202003_cli_fec.loc[pasivo_201803_202003_cli_fec['periodo']==int(date), 'id_cliente'].unique()
    
    if ix == 0:
        pasv_out = gp_func(ids, date)
    else:    
        pasv_temp = gp_func(ids, date)
        pasv_out = pd.concat([pasv_out, pasv_temp], ignore_index=True)

        
# ## Set_index
# pasivo_201803_202003_cli_fec.set_index(['id_cliente','periodo'], inplace=True)
# pasv_out.set_index(['id_cliente','periodo'], inplace=True)

202012
202101
202102


In [52]:
## Set_index
pasivo_201803_202003_cli_fec.set_index(['id_cliente','periodo'], inplace=True)
pasv_out.set_index(['id_cliente','periodo'], inplace=True)

In [53]:
pasivo_201803_202003_cli_fec_tot = pasivo_201803_202003_cli_fec.join(pasv_out, how='left')
print(pasv_out.shape)
print(pasivo_201803_202003_cli_fec.shape)
print(pasivo_201803_202003_cli_fec_tot.shape)

pasivo_201803_202003_cli_fec_tot.head()

(1847224, 2)
(1847224, 8)
(1847224, 10)


SMLV  fecha_pasivo  pasv_antig_total  \
id_cliente         periodo                                             
101052294083942902 202012   877803.0        202012        177.300000   
                   202101   908526.0        202101        178.333333   
                   202102   908526.0        202102        179.366667   
101052314032556001 202012   877803.0        202012          3.600000   
                   202101   908526.0        202101          4.633333   

                            pasv_num_meses_ult_apertura  \
id_cliente         periodo                                
101052294083942902 202012                    177.300000   
                   202101                    178.333333   
                   202102                    179.366667   
101052314032556001 202012                      3.600000   
                   202101                      4.633333   

                            pasv_dias_desde_ultima_trans  \
id_cliente         periodo                                 
101052294083942902 202012                         3975.0   
                   202101                         4006.0   
                   202102                         4037.0   
101052314032556001 202012                           84.0   
                   202101                          115.0   

                            pasv_saldo_tot_fin_mes_smlv  \
id_cliente         periodo                                
101052294083942902 202012                      0.000000   
                   202101                      0.000000   
                   202102                      0.000000   
101052314032556001 202012                      0.004266   
                   202101                      0.004122   

                            pasv_saldo_ca_fin_mes_smlv      fecha  \
id_cliente         periodo                                          
101052294083942902 202012                     0.000000 2020-12-01   
                   202101                     0.000000 2021-01-01   
                   202102                     0.000000 2021-02-01   
101052314032556001 202012                     0.004266 2020-12-01   
                   202101                     0.004122 2021-01-01   

                            pasv_saldo_total_1mes_atras  \
id_cliente         periodo                                
101052294083942902 202012                      0.000000   
                   202101                      0.000000   
                   202102                      0.000000   
101052314032556001 202012                      0.004266   
                   202101                      0.004122   

                            pasv_saldo_ca_1mes_atras  
id_cliente         periodo                            
101052294083942902 202012                   0.000000  
                   202101                   0.000000  
                   202102                   0.000000  
101052314032556001 202012                   0.004266  
                   202101                   0.004122

In [54]:
pasivo_201803_202003_cli_fec_tot.reset_index(inplace=True)

In [55]:
pasivo_201803_202003_cli_fec_tot = pasivo_201803_202003_cli_fec_tot.drop(['SMLV','periodo','fecha','pasv_saldo_ca_fin_mes_smlv','pasv_saldo_tot_fin_mes_smlv'], axis=1)

In [56]:
pasivo_201803_202003_cli_fec_tot.tail(50)

,id_cliente,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
1847174,999656652159656602,202101,133.400000,1.000000,6.0,82.256842,51.437691
1847175,999656652159656602,202102,134.433333,2.033333,13.0,89.457949,58.638798
1847176,999755260498918701,202012,64.233333,64.233333,3.0,3.587921,3.587921
1847177,999755260498918701,202101,65.266667,65.266667,12.0,2.857098,2.857098
1847178,999755260498918701,202102,66.300000,66.300000,9.0,2.502012,2.502012
1847179,999755260616443801,202012,52.500000,52.500000,12.0,1.148218,1.148218
1847180,999755260616443801,202101,53.533333,53.533333,18.0,1.127261,1.127261
1847181,999755260616443801,202102,54.566667,54.566667,15.0,0.047812,0.047812
1847182,999755266321666002,202012,132.800000,132.800000,8.0,0.005762,0.005762
1847183,999755266321666002,202101,133.833333,133.833333,11.0,0.242750,0.242750


In [57]:
pasivo_201803_202003_cli_fec_tot.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pasivos_M"+str(periodo),engine='pyarrow', index=False)

In [58]:
prueba = pasivo_201803_202003_cli_fec_tot[pasivo_201803_202003_cli_fec_tot['fecha_pasivo'] == periodo]

In [59]:
prueba.head()

,id_cliente,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
2,101052294083942902,202102,179.366667,179.366667,4037.0,0.000000,0.000000
5,101052314032556001,202102,5.666667,5.666667,146.0,0.004122,0.004122
8,101052315812399301,202102,61.600000,61.600000,51.0,0.006042,0.006042
11,101055253543395201,202102,175.633333,175.633333,9.0,16.740208,16.740208
14,101055260650910902,202102,137.600000,137.600000,147.0,0.000000,0.000000


In [60]:
pasivo_201803_202003_cli_fec_tot['fecha_pasivo'].value_counts().sort_index()

202012    620143
202101    614666
202102    612415
Name: fecha_pasivo, dtype: int64